In [126]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

cuda:0


In [127]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from catboost import CatBoostClassifier

from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
import lightgbm as lgb
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import GridSearchCV, StratifiedKFold

import warnings

# 경고 무시
warnings.filterwarnings("ignore", category=FutureWarning)

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler




import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR


In [128]:
train = pd.read_csv("../data/train_preprocessing.csv")
test = pd.read_csv("../data/test_preprocessing.csv")
sample_submission_id = pd.read_csv("../data/sample_submission.csv")['ID'].values

In [129]:
train

,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급
0,12480000,0,3.0,3,18.092177,2.990720,2.772589,1,0.000000,0.000000,0.000000,0.0,0.0,2
1,14400000,1,4.0,1,18.689180,3.149740,3.091042,10,0.000000,12.830869,12.363337,0.0,0.0,1
2,12000000,0,2.0,1,18.379859,2.261763,2.708050,1,0.000000,13.741482,11.931274,0.0,0.0,0
3,14400000,0,3.0,1,18.698312,2.778198,2.772589,1,0.000000,12.694116,11.938905,0.0,0.0,2
4,18000000,1,0.0,3,18.088503,3.272985,2.995732,8,0.000000,12.339471,11.911413,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96289,14400000,0,4.0,1,19.162618,2.335052,3.526361,3,0.000000,13.789763,13.106577,0.0,0.0,2
96290,28800000,1,4.0,1,18.698312,1.818077,3.258097,10,0.000000,13.277192,13.658956,0.0,0.0,4
96291,14400000,0,1.0,1,18.246327,2.504709,3.135494,3,0.000000,14.213702,12.393535,0.0,0.0,0
96292,15600000,0,2.0,1,18.010153,2.906901,3.091042,1,1.098612,14.136411,13.614712,0.0,0.0,3


In [130]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96294 entries, 0 to 96293
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   대출금액          96294 non-null  int64  
 1   대출기간          96294 non-null  int64  
 2   근로기간          96294 non-null  float64
 3   주택소유상태        96294 non-null  int64  
 4   연간소득          96294 non-null  float64
 5   부채_대비_소득_비율   96294 non-null  float64
 6   총계좌수          96294 non-null  float64
 7   대출목적          96294 non-null  int64  
 8   최근_2년간_연체_횟수  96294 non-null  float64
 9   총상환원금         96294 non-null  float64
 10  총상환이자         96294 non-null  float64
 11  총연체금액         96294 non-null  float64
 12  연체계좌수         96294 non-null  float64
 13  대출등급          96294 non-null  int64  
dtypes: float64(9), int64(5)
memory usage: 10.3 MB


In [131]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64197 entries, 0 to 64196
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   대출금액          64197 non-null  int64  
 1   대출기간          64197 non-null  int64  
 2   근로기간          64197 non-null  float64
 3   주택소유상태        64197 non-null  int64  
 4   연간소득          64197 non-null  float64
 5   부채_대비_소득_비율   64197 non-null  float64
 6   총계좌수          64197 non-null  float64
 7   대출목적          64197 non-null  int64  
 8   최근_2년간_연체_횟수  64197 non-null  float64
 9   총상환원금         64197 non-null  float64
 10  총상환이자         64197 non-null  float64
 11  총연체금액         64197 non-null  float64
 12  연체계좌수         64197 non-null  float64
dtypes: float64(9), int64(4)
memory usage: 6.4 MB


In [132]:

# 인코더를 초기화할 때 handle_unknown='ignore' 옵션 추가
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

# train 데이터셋에 대해 fit_transform을 적용
encoded_train = encoder.fit_transform(train[['주택소유상태','대출목적','근로기간']])
columns_train = encoder.get_feature_names_out(['주택소유상태','대출목적','근로기간'])

# 원-핫 인코딩된 데이터프레임 생성
one_hot_encoded_train_df = pd.DataFrame(encoded_train, columns=columns_train)

# test 데이터셋에 대해서는 transform을 적용
encoded_test = encoder.transform(test[['주택소유상태','대출목적','근로기간']])

# 새로운 컬럼 이름 생성
columns_test = encoder.get_feature_names_out(['주택소유상태','대출목적','근로기간'])

# 원-핫 인코딩된 데이터프레임 생성
one_hot_encoded_test_df = pd.DataFrame(encoded_test, columns=columns_train)

# 원본 train 데이터프레임과 인코딩된 데이터프레임을 합치기
train_without_columns = train.drop(columns=['주택소유상태', '대출목적', '근로기간'])
one_hot_encoded_train_df = pd.concat([train_without_columns.reset_index(drop=True), one_hot_encoded_train_df], axis=1)

# 원본 test 데이터프레임과 인코딩된 데이터프레임을 합치기
test_without_columns = test.drop(columns=['주택소유상태', '대출목적', '근로기간'])
one_hot_encoded_test_df = pd.concat([test_without_columns.reset_index(drop=True), one_hot_encoded_test_df], axis=1)

train = one_hot_encoded_train_df
test = one_hot_encoded_test_df

In [133]:
# train = train[train['최근_2년간_연체_횟수']<2.5]
# train = train[train['총연체금액']<1]
# train = train[train['부채_대비_소득_비율']<6]

In [134]:
# MinMaxScaler 초기화
scaler = MinMaxScaler()

# '대출등급' 열을 제외하고 train 데이터에 스케일링 적용
train_columns = train.columns.drop('대출등급')
train_scaled = scaler.fit_transform(train[train_columns])

# 스케일링된 train 데이터를 DataFrame으로 변환하고 '대출등급' 열 추가
train_scaled = pd.DataFrame(train_scaled, columns=train_columns, index=train.index)
train_scaled['대출등급'] = train['대출등급']

# 동일한 scaler를 사용하여 test 데이터에 스케일링 적용
test_scaled = scaler.transform(test)

# 스케일링된 test 데이터를 DataFrame으로 변환
test = pd.DataFrame(test_scaled, columns=test.columns)

# train 데이터 업데이트
train = train_scaled

In [135]:
train

,대출금액,대출기간,연간소득,부채_대비_소득_비율,총계좌수,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,...,대출목적_8,대출목적_9,대출목적_10,대출목적_11,근로기간_0.0,근로기간_1.0,근로기간_2.0,근로기간_3.0,근로기간_4.0,대출등급
0,0.276471,0.0,0.783116,0.324713,0.329845,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2
1,0.323529,1.0,0.808957,0.341979,0.420151,0.000000,0.731015,0.795184,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1
2,0.264706,0.0,0.795568,0.245568,0.311543,0.000000,0.782895,0.767395,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
3,0.323529,0.0,0.809352,0.301639,0.329845,0.000000,0.723224,0.767885,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2
4,0.411765,1.0,0.782957,0.355360,0.393123,0.000000,0.703018,0.766117,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96289,0.323529,0.0,0.829450,0.253525,0.543598,0.000000,0.785646,0.842988,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2
96290,0.676471,1.0,0.809352,0.197395,0.467524,0.000000,0.756443,0.878515,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,4
96291,0.323529,0.0,0.789788,0.271945,0.432757,0.000000,0.809799,0.797126,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
96292,0.352941,0.0,0.779565,0.315613,0.420151,0.319923,0.805396,0.875670,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3


In [136]:
test

,대출금액,대출기간,연간소득,부채_대비_소득_비율,총계좌수,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,...,대출목적_7,대출목적_8,대출목적_9,대출목적_10,대출목적_11,근로기간_0.0,근로기간_1.0,근로기간_2.0,근로기간_3.0,근로기간_4.0
0,0.382353,0.0,0.809352,0.328677,0.270962,0.000000,0.734148,0.765093,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.176471,0.0,0.792761,0.306586,0.467524,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.394118,0.0,0.814885,0.243397,0.406959,0.000000,0.820188,0.807127,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.323529,0.0,0.779350,0.291977,0.517403,0.201849,0.764214,0.807105,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.647059,0.0,0.771615,0.374578,0.270962,0.000000,0.799829,0.857301,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64192,0.705882,0.0,0.786580,0.340809,0.488540,0.319923,0.802390,0.871219,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
64193,0.705882,1.0,0.801145,0.278986,0.478226,0.000000,0.784824,0.902692,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
64194,0.120588,0.0,0.757239,0.368554,0.543598,0.000000,0.671538,0.726840,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
64195,0.252941,0.0,0.779350,0.355565,0.603521,0.201849,0.803768,0.855930,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [137]:
train_x = train.drop('대출등급', axis=1)
train_y = train['대출등급']
test_x = test

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2, random_state=43)

In [138]:
from torch.utils.data import TensorDataset, DataLoader

# X_train, X_test, y_train, y_test를 PyTorch Tensor로 변환
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)
real_test_tensor = torch.tensor(test_x.values, dtype=torch.float32)

# TensorDataset과 DataLoader 생성
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)

real_test_dataset = TensorDataset(real_test_tensor)
real_test_loader = DataLoader(real_test_dataset, batch_size=1024, shuffle=False)

In [139]:
import lovely_tensors as lt
lt.monkey_patch()
X_train_tensor

tensor[77035, 31] n=2388085 (9.1Mb) x∈[0., 1.000] μ=0.220 σ=0.368

In [140]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.layer1 = nn.Linear(X_train.shape[1], 64)
        self.bn1 = nn.BatchNorm1d(64)  # 첫 번째 레이어의 배치 정규화
        self.layer2 = nn.Linear(64, 128)
        self.bn2 = nn.BatchNorm1d(128)  # 두 번째 레이어의 배치 정규화
        self.layer3 = nn.Linear(128, 256)
        self.bn3 = nn.BatchNorm1d(256)  # 세 번째 레이어의 배치 정규화
        self.layer4 = nn.Linear(256, 512)
        self.bn4 = nn.BatchNorm1d(512)   # 네 번째 레이어의 배치 정규화
        self.layer5 = nn.Linear(512,1024)
        self.bn5 = nn.BatchNorm1d(1024)   # 다섯 번째 레이어의 배치 정규화
        self.layer6 = nn.Linear(1024,256)
        self.bn6 = nn.BatchNorm1d(256)   # 다섯 번째 레이어의 배치 정규화
        self.layer7 = nn.Linear(256, 64)
        self.bn7 = nn.BatchNorm1d(64)   # 다섯 번째 레이어의 배치 정규화
        self.output_layer = nn.Linear(64, 7)  # 7개 클래스에 대한 출력

    def forward(self, x):
        x = torch.relu(self.bn1(self.layer1(x)))
        x = torch.relu(self.bn2(self.layer2(x)))
        x = torch.relu(self.bn3(self.layer3(x)))
        x = torch.relu(self.bn4(self.layer4(x)))
        x = torch.relu(self.bn5(self.layer5(x)))
        x = torch.relu(self.bn6(self.layer6(x)))
        x = torch.relu(self.bn7(self.layer7(x)))
        x = self.output_layer(x)
        return x
    
def get_lr(opt):
    for param_group in opt.param_groups:
        return param_group['lr']
    
# 모델, 손실 함수, 옵티마이저 초기화
model = NeuralNet()
model = model.to(device)
criterion = nn.CrossEntropyLoss()

In [148]:
# optimizer = optim.Adam(model.parameters(), lr=0.003)
optimizer = optim.AdamW(model.parameters(), lr=0.003)
lr_scheduler = StepLR(optimizer, step_size=1, gamma=0.95)

# 학습 및 검증 과정
# Early stopping을 위한 설정
patience = 20  # 성능 개선이 없는 에포크 수
best_f1 = 0.0
counter = 0
num_epochs = 100  # 에포크 수 설정
for epoch in range(num_epochs):
    # 학습 과정
    current_lr = get_lr(optimizer)
    print('Epoch {}/{}, current lr={}'.format(epoch+1,num_epochs, current_lr))
    model.train()
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)  # 데이터를 GPU로 이동
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    lr_scheduler.step()

    # 검증 과정
    model.eval()
    total = 0
    correct = 0
    all_predicted = []
    all_targets = []
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
            all_predicted.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    accuracy = 100 * correct / total
    f1_macro = f1_score(all_targets, all_predicted, average='macro')
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Validation Accuracy: {accuracy:.2f}%, F1 Macro: {f1_macro:.4f}')

    # Early stopping 체크
    if f1_macro > best_f1:
        best_f1 = f1_macro
        counter = 0
        torch.save(model.state_dict(), '../data/best_weight.pth')  # 모델 가중치 저장
        print(f'Model saved at epoch {epoch+1}')
    else:
        counter += 1
        if counter >= patience:
            print(f'Early stopping triggered at epoch {epoch+1}')
            break

Epoch 1/100, current lr=0.003
Epoch 1/100, Loss: 0.6297, Validation Accuracy: 61.44%, F1 Macro: 0.5300
Model saved at epoch 1
Epoch 2/100, current lr=0.00285
Epoch 2/100, Loss: 0.6570, Validation Accuracy: 53.63%, F1 Macro: 0.3888
Epoch 3/100, current lr=0.0027075
Epoch 3/100, Loss: 0.5589, Validation Accuracy: 57.54%, F1 Macro: 0.4487
Epoch 4/100, current lr=0.0025721249999999998
Epoch 4/100, Loss: 0.5558, Validation Accuracy: 75.55%, F1 Macro: 0.6272
Model saved at epoch 4
Epoch 5/100, current lr=0.0024435187499999996
Epoch 5/100, Loss: 0.5475, Validation Accuracy: 68.15%, F1 Macro: 0.6082
Epoch 6/100, current lr=0.0023213428124999997
Epoch 6/100, Loss: 0.5181, Validation Accuracy: 74.29%, F1 Macro: 0.6948
Model saved at epoch 6
Epoch 7/100, current lr=0.0022052756718749997
Epoch 7/100, Loss: 0.5948, Validation Accuracy: 71.22%, F1 Macro: 0.6873
Epoch 8/100, current lr=0.0020950118882812497
Epoch 8/100, Loss: 0.5161, Validation Accuracy: 76.97%, F1 Macro: 0.6753
Epoch 9/100, current 

KeyboardInterrupt: 

In [151]:
model.eval()
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()
        all_predicted.extend(predicted.cpu().numpy())
        all_targets.extend(targets.cpu().numpy())

accuracy = 100 * correct / total
f1_macro = f1_score(all_targets, all_predicted, average='macro')
print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Validation Accuracy: {accuracy:.2f}%, F1 Macro: {f1_macro:.4f}')

Epoch 25/100, Loss: 0.4250, Validation Accuracy: 81.94%, F1 Macro: 0.7762


# Prediciton

In [150]:
model.load_state_dict(torch.load('../data/best_weight.pth'))

<All keys matched successfully>

In [152]:
model.eval()  # 모델을 평가 모드로 설정
predictions = []

with torch.no_grad():
    for inputs in real_test_loader:  # 레이블 데이터 없음
        inputs = inputs[0].to(device)  # 첫 번째 원소가 입력 데이터
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())

In [153]:
pred_final = np.array(predictions)

In [154]:
submit = pd.DataFrame({
    'ID': sample_submission_id,
    '대출등급': pred_final.flatten()
})

In [155]:
submit

,ID,대출등급
0,TEST_00000,2
1,TEST_00001,3
2,TEST_00002,0
3,TEST_00003,3
4,TEST_00004,2
...,...,...
64192,TEST_64192,2
64193,TEST_64193,4
64194,TEST_64194,3
64195,TEST_64195,4


In [122]:
# 숫자를 문자로 매핑할 딕셔너리를 생성합니다.
num_to_letter = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G'}

# '대출등급' 컬럼에 map 함수를 적용하여 숫자를 문자로 변환합니다.
submit['대출등급'] = submit['대출등급'].map(num_to_letter)

In [304]:
submit.to_csv("../submission/test5_deep.csv", index = False)